In [4]:
!pip install langchain faiss-cpu sentence-transformers huggingface-hub transformers langchain-community llama-cpp-python


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("test2.txt",encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
len(chunks)

d:\final Year project\Research_backend\RAG\venvRAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


7

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True}
)


db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.6
    }
)



C:\Users\mathivathanan\AppData\Local\Temp\ipykernel_14068\1233062235.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [26]:
from langchain_community.llms import LlamaCpp


llm = LlamaCpp(
    model_path=r"D:\final Year project\Research_backend\LLM_Talk_Back\models\tamil-llama-7b-v0.1-q8_0.gguf",
    temperature=0.2,
    max_tokens=256,
    n_ctx=4096,
    verbose=False
)


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


In [27]:
def rag_chain(question):
   
    # 3. Format the prompt
    prompt_text = f"""


கொடுக்கப்பட்ட பந்தியையும் சுருக்கமாக எழுதி, அதற்கான சிறிய தலைப்பையும் வழங்கவும்

செய்தி:

1.உள்ளூர் வளர்ச்சி திட்டங்களில் முன்னேற்றம்

ஸ்கேன் செய்யப்பட்ட செய்தித்தாள் தகவல்களின் அடிப்படையில், உள்ளூர் வளர்ச்சி திட்டங்களில் சமீப காலமாக கணிசமான முன்னேற்றம் ஏற்பட்டுள்ளதாக தெரிவிக்கப்படுகிறது. சாலைப் பராமரிப்பு, குடிநீர் வசதி மற்றும் பொது போக்குவரத்து சேவைகள் மேம்படுத்தப்பட்டுள்ளன. பொதுமக்களின் அடிப்படை தேவைகளை பூர்த்தி செய்யும் வகையில் இந்த திட்டங்கள் செயல்படுத்தப்படுவதால், வாழ்வாதாரத் தரம் மேம்படும் என அதிகாரிகள் நம்பிக்கை தெரிவித்துள்ளனர்.

2.கல்வித் துறையில் புதிய மாற்றங்கள்

செய்தித்தாள் பக்கத்தில் இடம்பெற்ற தகவல்களின்படி, கல்வித் துறையில் பல புதிய மாற்றங்கள் நடைமுறைக்கு வந்துள்ளன. பள்ளிகளில் டிஜிட்டல் கற்றல் முறைகள் அறிமுகப்படுத்தப்பட்டு, ஆசிரியர்களுக்கான பயிற்சிகள் நடத்தப்பட்டுள்ளன. இதன் மூலம் மாணவர்களின் கற்றல் திறன் அதிகரிக்கும் என்றும், தொழில்நுட்ப அறிவு வளர்க்கப்படும் என்றும் தெரிவிக்கப்பட்டுள்ளது.

3.பொருளாதார நிலை குறித்து மதிப்பீடு

ஸ்கேன் செய்யப்பட்ட செய்திகளில் பொருளாதார நிலை குறித்த விரிவான மதிப்பீடுகள் இடம்பெற்றுள்ளன. விலை நிலவரம், வேலை வாய்ப்புகள் மற்றும் வர்த்தக நடவடிக்கைகள் குறித்து ஆராய்ந்து, பொருளாதாரம் மெல்ல மெல்ல நிலைபெறுகிறது என்று நிபுணர்கள் கருத்து தெரிவித்துள்ளனர். சிறு மற்றும் நடுத்தர தொழில்கள் மீண்டும் செயல்படத் தொடங்கியுள்ளன என்பதும் குறிப்பிடப்பட்டுள்ளது.

4.சமூக ஒற்றுமை மற்றும் பொதுமக்கள் பங்கேற்பு

செய்தித்தாள் அறிக்கைகளின் படி, சமூக ஒற்றுமை மற்றும் பொதுமக்கள் பங்கேற்பு அதிகரித்து வருகிறது. சுத்தம், சுற்றுச்சூழல் பாதுகாப்பு மற்றும் சமூக சேவை நடவடிக்கைகளில் மக்கள் தன்னார்வமாக ஈடுபடுகின்றனர். இந்த ஒத்துழைப்பு சமூக வளர்ச்சிக்கு வலுவான அடித்தளமாக அமையும் என்று கருத்து தெரிவிக்கப்பட்டுள்ளது.


கேள்வி: 

"""

    # 4. Call the model
    raw = llm.invoke(prompt_text)
    answer = raw.replace(prompt_text, "").strip()
    print(prompt_text)
    return answer

In [8]:
def rag_chain(question):
    # 1. Retrieve similar docs
    docs = retriever.invoke(question)

    if not docs:
        return "நீங்கள் கேட்ட கேள்விக்கான பதில், இங்கு உள்ள தகவல்களில் இல்லை. இணையத்தில் தேடி பதில் சொல்ல வேண்டுமா?"


    # 2. Combine the doc text
    context = "\n\n".join([d.page_content for d in docs])

    # 3. Format the prompt
    prompt_text = f"""

கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
{context}

கேள்வி: {question}

"""

    # 4. Call the model
    raw = llm.invoke(prompt_text)
    answer = raw.replace(prompt_text, "").strip()
    print(prompt_text)
    return answer

In [28]:
response = rag_chain("நகரமயமாக்கல் வேகமாக வளர்வதற்கான முக்கிய காரணம் என்ன?")
print(response)






கொடுக்கப்பட்ட பந்தியையும் சுருக்கமாக எழுதி, அதற்கான சிறிய தலைப்பையும் வழங்கவும்

செய்தி:

1.உள்ளூர் வளர்ச்சி திட்டங்களில் முன்னேற்றம்

ஸ்கேன் செய்யப்பட்ட செய்தித்தாள் தகவல்களின் அடிப்படையில், உள்ளூர் வளர்ச்சி திட்டங்களில் சமீப காலமாக கணிசமான முன்னேற்றம் ஏற்பட்டுள்ளதாக தெரிவிக்கப்படுகிறது. சாலைப் பராமரிப்பு, குடிநீர் வசதி மற்றும் பொது போக்குவரத்து சேவைகள் மேம்படுத்தப்பட்டுள்ளன. பொதுமக்களின் அடிப்படை தேவைகளை பூர்த்தி செய்யும் வகையில் இந்த திட்டங்கள் செயல்படுத்தப்படுவதால், வாழ்வாதாரத் தரம் மேம்படும் என அதிகாரிகள் நம்பிக்கை தெரிவித்துள்ளனர்.

2.கல்வித் துறையில் புதிய மாற்றங்கள்

செய்தித்தாள் பக்கத்தில் இடம்பெற்ற தகவல்களின்படி, கல்வித் துறையில் பல புதிய மாற்றங்கள் நடைமுறைக்கு வந்துள்ளன. பள்ளிகளில் டிஜிட்டல் கற்றல் முறைகள் அறிமுகப்படுத்தப்பட்டு, ஆசிரியர்களுக்கான பயிற்சிகள் நடத்தப்பட்டுள்ளன. இதன் மூலம் மாணவர்களின் கற்றல் திறன் அதிகரிக்கும் என்றும், தொழில்நுட்ப அறிவு வளர்க்கப்படும் என்றும் தெரிவிக்கப்பட்டுள்ளது.

3.பொருளாதார நிலை குறித்து மதிப்பீடு

ஸ்கேன் செய்யப்பட்ட செய்திகளில் பொருளாதார 

In [10]:
response = rag_chain("தமிழ்நாட்டின் தலைநகர் எது?")
print(response)

No relevant docs were retrieved using the relevance score threshold 0.6


நீங்கள் கேட்ட கேள்விக்கான பதில், இங்கு உள்ள தகவல்களில் இல்லை. இணையத்தில் தேடி பதில் சொல்ல வேண்டுமா?
